# **Access Drive File**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Behavioral Assessments Dataset**

# **Preprocessing**

# **Text-to-Embeddings Conversion Using BERT**

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
import nltk
import re
import string
from nltk.corpus import stopwords

# -----------------------------------
# 0. Download NLTK stopwords
# -----------------------------------
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# -----------------------------------
# 1. Preprocessing function
# -----------------------------------
def preprocess_text(text):
    """Clean and normalize text data"""
    if not isinstance(text, str):
        return ""
    text = text.lower()  # lowercase
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # remove unwanted chars
    tokens = [word for word in text.split() if word not in stop_words]  # remove stopwords
    return " ".join(tokens)

# -----------------------------------
# 2. Load BERT model + tokenizer
# -----------------------------------
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# -----------------------------------
# 3. Embedding function (Eqns 3–6)
# -----------------------------------
def get_bert_embeddings(text, pooling="cls"):
    """Convert text into embeddings using BERT"""
    clean_text = preprocess_text(text)
    inputs = tokenizer(clean_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = bert_model(**inputs)

    last_hidden_state = outputs.last_hidden_state.squeeze(0)  # (seq_len, hidden_dim)
    cls_embedding = outputs.pooler_output.squeeze(0)          # CLS token embedding

    if pooling == "cls":
        return cls_embedding.numpy()
    elif pooling == "mean":
        return last_hidden_state.mean(dim=0).numpy()
    else:
        raise ValueError("Pooling must be 'cls' or 'mean'")

# -----------------------------------
# 4. Fusion with EEG embeddings (Eqn 7)
# -----------------------------------
def fuse_embeddings(eeg_embedding, text_embedding, w1=0.5, w2=0.5, bias=0.1):
    eeg_part = w1 * np.array(eeg_embedding)
    text_part = w2 * np.array(text_embedding) + bias
    return np.concatenate([eeg_part, text_part])

# -----------------------------------
# 5. Process CSV input
# -----------------------------------
def process_csv(input_csv, text_column, output_csv, pooling="cls", use_fusion=False, keep_columns=None):
    """
    input_csv: path to input CSV file
    text_column: name of the column containing text
    output_csv: path to save embeddings + label column
    pooling: 'cls' or 'mean'
    use_fusion: whether to fuse with EEG embeddings
    keep_columns: list of columns from the input CSV to also include in the output
    """
    df = pd.read_csv(input_csv)

    embeddings = []
    for idx, text in enumerate(df[text_column]):
        emb = get_bert_embeddings(str(text), pooling=pooling)

        if use_fusion:
            eeg_emb = np.random.rand(len(emb))  # Simulated EEG embedding
            emb = fuse_embeddings(eeg_emb, emb)

        embeddings.append(emb)

    # Convert list of embeddings to DataFrame
    emb_df = pd.DataFrame(embeddings)

    # Add original column(s) back
    if keep_columns is None:
        emb_df[text_column] = df[text_column]
    else:
        for col in keep_columns:
            emb_df[col] = df[col]

    # Save to CSV
    emb_df.to_csv(output_csv, index=False)
    print(f"✅ Saved embeddings + labels to {output_csv}")

# -----------------------------------
# Example usage
# -----------------------------------
if __name__ == "__main__":
    input_csv = "/content/drive/MyDrive/Colab Notebooks/archive (22)/Dataset-Mental-Disorders.csv"
    output_csv = "/content/drive/MyDrive/Colab Notebooks/archive (22)/text_embeddings.csv"
    text_column = "Expert Diagnose"   # column containing text

    # Keep additional output columns (optional)
    keep_cols = ["Expert Diagnose"]   # or e.g. ["Expert Diagnose", "Patient ID", "Age"]

    # Process CSV → Save embeddings
    process_csv(input_csv, text_column, output_csv, pooling="cls", use_fusion=True, keep_columns=keep_cols)

    # Load and check
    df = pd.read_csv(output_csv)
    print(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Saved embeddings + labels to /content/drive/MyDrive/Colab Notebooks/archive (22)/text_embeddings.csv
          0         1         2         3         4         5         6  \
0  0.163934  0.020747  0.053941  0.389782  0.453994  0.055306  0.424364   
1  0.221380  0.264213  0.396977  0.465149  0.101821  0.118971  0.093329   
2  0.409628  0.439373  0.326223  0.442865  0.024248  0.016242  0.158557   
3  0.107815  0.043820  0.388044  0.189001  0.481159  0.419166  0.469136   
4  0.339679  0.495903  0.433516  0.274738  0.421538  0.043065  0.020213   

          7         8         9  ...      1527      1528      1529      1530  \
0  0.068270  0.478556  0.157259  ...  0.230639  0.217148  0.502674  0.181111   
1  0.010445  0.181457  0.211408  ...  0.166952  0.390829  0.520842  0.036960   
2  0.154312  0.349947  0.482482  ...  0.230639  0.217148  0.502674  0.181111   
3  0.273275  0.336277  0.117473  ...  0.230639  0.217148  0.502674  0.181111   
4  0.013070  0.137461  0.242913  ...  0.234224

# **Feature extraction**

# **BiLSTM-CNN Algorithm**

In [ ]:
# -----------------------------
# Import Libraries
# -----------------------------
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# -----------------------------
# 1. Load CSV
# -----------------------------
csv_file = "/content/drive/MyDrive/Colab Notebooks/archive (22)/text_embeddings.csv"
df = pd.read_csv(csv_file)

# Separate features and labels
y = df['Expert Diagnose'].values
X = df.drop('Expert Diagnose', axis=1).values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -----------------------------
# 2. Infer number of channels and time steps
# -----------------------------
num_channels = 14  # adjust for your EEG
total_features = X.shape[1]

if total_features % num_channels != 0:
    time_steps = total_features // num_channels
    X = X[:, :num_channels*time_steps]
else:
    time_steps = total_features // num_channels

print(f"Num channels: {num_channels}, Time steps: {time_steps}, X shape: {X.shape}")

X = X.reshape(-1, num_channels, time_steps)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# -----------------------------
# 3. EEG Dataset Class
# -----------------------------
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = EEGDataset(X, y_encoded)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# -----------------------------
# 4. BiLSTM-CNN Model (Feature Extractor)
# -----------------------------
class BiLSTM_CNN_FeatureExtractor(nn.Module):
    def __init__(self, input_channels, time_steps, hidden_dim, cnn_out_channels):
        super(BiLSTM_CNN_FeatureExtractor, self).__init__()

        # BiLSTM
        self.bilstm = nn.LSTM(input_size=input_channels, hidden_size=hidden_dim,
                              num_layers=1, batch_first=True, bidirectional=True)

        # CNN
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)

    def forward(self, x):
        # BiLSTM features
        x_lstm = x.permute(0, 2, 1)  # (batch, time_steps, channels)
        lstm_out, _ = self.bilstm(x_lstm)
        lstm_feat = lstm_out[:, -1, :]  # last time step

        # CNN features
        cnn_out = self.conv1(x)
        cnn_out = self.relu(cnn_out)
        cnn_out = self.pool(cnn_out)
        cnn_feat = cnn_out.view(cnn_out.size(0), -1)

        # Concatenate features
        fused_feat = torch.cat((lstm_feat, cnn_feat), dim=1)
        return fused_feat

hidden_dim = 64
cnn_out_channels = 32
model = BiLSTM_CNN_FeatureExtractor(input_channels=num_channels, time_steps=time_steps,
                                    hidden_dim=hidden_dim, cnn_out_channels=cnn_out_channels)

model.eval()

# -----------------------------
# 5. Extract Features and Save to CSV
# -----------------------------
all_features = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in loader:
        feats = model(X_batch)
        all_features.append(feats.numpy())
        all_labels.append(y_batch.numpy())

all_features = np.vstack(all_features)
all_labels = np.concatenate(all_labels)

# Combine features and labels
df_features = pd.DataFrame(all_features)
df_features['Expert Diagnose'] = all_labels

# Save to CSV
output_csv = "/content/drive/MyDrive/Colab Notebooks/archive (22)/extraction_features.csv"
df_features.to_csv(output_csv, index=False)
print(f"Feature extraction completed. Saved to {output_csv}")

df_features

Num channels: 14, Time steps: 109, X shape: (120, 1526)
Feature extraction completed. Saved to /content/drive/MyDrive/Colab Notebooks/archive (22)/extraction_features.csv


,0,1,2,3,4,5,6,7,8,9,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,Expert Diagnose
0,-0.023460,-0.009872,0.082980,0.052413,0.006048,0.141314,0.243438,-0.079263,0.024943,0.031178,...,0.171451,0.000000,0.315461,0.000000,0.028471,0.535123,0.723172,0.000000,0.000000,1
1,-0.084386,-0.103192,-0.088962,0.030697,-0.028180,0.006336,-0.020375,-0.022547,0.086264,0.062180,...,0.564366,0.000000,0.719081,0.366801,0.707353,0.582269,0.000000,0.654189,1.796574,2
2,-0.018671,-0.069374,0.078253,-0.088370,0.020094,0.086396,0.116505,-0.109338,0.081247,-0.068201,...,0.000000,0.675273,0.000000,0.044222,0.000000,0.000000,0.000000,0.258092,0.000000,0
3,0.027535,-0.089727,0.029530,0.006031,0.069494,0.010698,0.075243,-0.070218,0.108728,-0.032648,...,0.892055,0.267894,0.000000,0.573140,0.000000,0.000000,0.574548,0.065988,0.000000,1
4,-0.083207,-0.132342,0.056536,-0.014271,-0.072003,0.103119,0.138146,-0.020543,-0.044462,-0.131839,...,0.831823,0.174909,0.318508,0.000000,0.471079,0.000000,0.023926,0.197720,0.390639,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,-0.112506,-0.043592,-0.112485,0.127277,-0.082939,0.006761,0.012738,0.029169,0.015405,0.079715,...,0.333294,0.429890,0.106210,0.365571,0.402822,0.287952,0.023290,0.104864,1.433301,2
116,-0.007931,-0.074937,0.070425,0.139553,0.022574,0.101328,0.171554,-0.066366,0.055047,0.035051,...,0.069739,0.000000,0.000000,0.886581,0.000000,0.000000,0.976803,0.156914,0.000000,0
117,-0.017736,-0.104074,-0.039863,0.014642,0.042931,0.074098,0.047254,-0.105637,0.084942,-0.068229,...,0.310899,0.000000,0.292860,0.218753,0.201924,0.000000,1.227339,0.220512,0.000000,1
118,-0.050813,0.036072,-0.107408,0.139934,-0.081417,0.059574,0.057207,0.059609,-0.022578,0.099347,...,0.000000,0.000000,0.240604,0.552566,0.734320,0.365220,0.000000,0.425974,1.785931,2


# **Feature selection via HAGWO**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# -----------------------------
# 1. Load CSV
# -----------------------------
csv_file = "/content/drive/MyDrive/Colab Notebooks/archive (22)/extraction_features.csv"
df = pd.read_csv(csv_file)

# Separate features and label
X = df.drop('Expert Diagnose', axis=1).values
y = df['Expert Diagnose'].values

# Optional: normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -----------------------------
# 2. HAGWO Class
# -----------------------------
class HAGWO:
    def __init__(self, X, y, num_ants=20, num_iterations=10, min_features=5):
        self.X = X
        self.y = y
        self.M = num_ants
        self.T = num_iterations
        self.N = X.shape[1]
        self.tau = np.ones(self.N)
        self.eta = np.random.rand(self.N)
        self.rho = 0.1
        self.q = 1.0
        self.min_features = min_features

    def fitness(self, subset):
        # Replace with real classifier if available
        if subset.sum() == 0:
            return 0
        return subset.sum() / len(subset)

    def aco_exploration(self):
        ant_subsets = []
        for _ in range(self.M):
            probs = (self.tau ** 1) * (self.eta ** 2)
            probs = probs / probs.sum()
            subset = np.random.rand(self.N) < probs
            # Ensure minimum features
            if subset.sum() < self.min_features:
                indices = np.random.choice(self.N, self.min_features, replace=False)
                subset[indices] = 1
            ant_subsets.append(subset.astype(int))
        return np.array(ant_subsets)

    def update_pheromones(self, ant_subsets):
        delta_tau = np.zeros(self.N)
        for subset in ant_subsets:
            fit = self.fitness(subset)
            delta_tau += self.q * subset * fit
        self.tau = (1 - self.rho) * self.tau + delta_tau

    def mgwo_exploitation(self, top_subsets):
        alpha, beta, delta = top_subsets[:3]
        new_subsets = []
        a = 2.0
        for subset in top_subsets:
            r1, r2, r3 = np.random.rand(3, self.N)
            D_alpha = np.abs(r1 * alpha - subset)
            D_beta = np.abs(r2 * beta - subset)
            D_delta = np.abs(r3 * delta - subset)
            x1 = alpha - a * D_alpha
            x2 = beta - a * D_beta
            x3 = delta - a * D_delta
            new_subset = (x1 + x2 + x3) / 3
            new_subset = (new_subset > 0.5).astype(int)
            # Ensure minimum features
            if new_subset.sum() < self.min_features:
                indices = np.random.choice(self.N, self.min_features, replace=False)
                new_subset[indices] = 1
            new_subsets.append(new_subset)
        return np.array(new_subsets)

    def select_features(self):
        best_subset = np.zeros(self.N)
        best_score = -np.inf

        for t in range(self.T):
            ant_subsets = self.aco_exploration()
            self.update_pheromones(ant_subsets)

            fitnesses = np.array([self.fitness(s) for s in ant_subsets])
            top_idx = fitnesses.argsort()[-max(1, self.M//10):][::-1]
            top_subsets = ant_subsets[top_idx]

            new_subsets = self.mgwo_exploitation(top_subsets)

            for subset in new_subsets:
                score = self.fitness(subset)
                if score > best_score:
                    best_score = score
                    best_subset = subset

        # Final check: ensure minimum features
        if best_subset.sum() < self.min_features:
            indices = np.random.choice(self.N, self.min_features, replace=False)
            best_subset[indices] = 1

        return best_subset

# -----------------------------
# 3. Run HAGWO
# -----------------------------
hagwo = HAGWO(X, y, num_ants=30, num_iterations=20, min_features=5)
selected_mask = hagwo.select_features()

# -----------------------------
# 4. Save reduced dataset with original column names
# -----------------------------
X_selected = X[:, selected_mask==1]
feature_columns = df.drop('Expert Diagnose', axis=1).columns
selected_columns = feature_columns[selected_mask==1]

df_selected = pd.DataFrame(X_selected, columns=selected_columns)
df_selected['Expert Diagnose'] = y  # Add label column

output_csv = "/content/drive/MyDrive/Colab Notebooks/archive (22)/selected_features.csv"
df_selected.to_csv(output_csv, index=False)

print(f"Selected features saved to {output_csv}")
print("Number of selected features:", X_selected.shape[1])
print("Selected feature columns:", list(selected_columns))
df_selected



Selected features saved to /content/drive/MyDrive/Colab Notebooks/archive (22)/selected_features.csv
Number of selected features: 5
Selected feature columns: ['141', '441', '982', '1362', '1452']


,141,441,982,1362,1452,Expert Diagnose
0,-1.220591,0.484499,0.505208,-0.688344,1.790367,1
1,-0.693150,-0.422360,-0.980420,-0.215150,-1.005319,2
2,-0.079629,-1.078309,1.270970,0.372264,1.387482,0
3,0.191726,0.396753,0.337527,1.026910,0.470842,1
4,0.802211,1.422706,-0.980420,2.941879,-1.039586,3
...,...,...,...,...,...,...
115,-1.220591,1.331524,-0.286117,-1.217429,-1.039586,2
116,1.304548,0.159161,1.204678,0.179233,0.649806,0
117,-0.056729,-1.078309,0.132382,0.230741,1.378096,1
118,-1.220591,-0.399138,-0.980420,-0.511834,-0.933408,2


# **Classification**

# **proposed Algorithm**

In [ ]:
"""
NeuroVisionNet: Multimodal mental health diagnosis model
- CSV numerical features as spatial + temporal inputs
- Label column: "Expert Diagnose"
- Prints training & validation accuracy
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ------------------------------
# Config
# ------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SPATIAL_FEATURE_DIM = 512
TEMPORAL_FEATURE_DIM = 256
FUSION_HIDDEN = 256
DROPOUT = 0.3
LR = 1e-4
BATCH_SIZE = 8
EPOCHS = 100

# ------------------------------
# Temporal branch: T-CNN (1D Conv)
# ------------------------------
class TemporalCNN(nn.Module):
    def __init__(self, in_channels=1, out_dim=TEMPORAL_FEATURE_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, 32, 7, padding=3), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 5, padding=2), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, out_dim)

    def forward(self, x):
        y = self.net(x)
        y = y.view(y.size(0), -1)
        return self.fc(y)

# ------------------------------
# NeuroVisionNet
# ------------------------------
class NeuroVisionNet(nn.Module):
    def __init__(self, input_dim, num_classes, dropout=DROPOUT):
        super().__init__()
        self.spatial = nn.Linear(input_dim, SPATIAL_FEATURE_DIM)
        self.temporal = TemporalCNN(out_dim=TEMPORAL_FEATURE_DIM)
        self.fusion = nn.Sequential(
            nn.Linear(SPATIAL_FEATURE_DIM + TEMPORAL_FEATURE_DIM, FUSION_HIDDEN),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(FUSION_HIDDEN, FUSION_HIDDEN//2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.classifier = nn.Linear(FUSION_HIDDEN//2, num_classes)

    def forward(self, spatial_feat, seq):
        f_spatial = self.spatial(spatial_feat)
        f_temporal = self.temporal(seq)
        f_combined = torch.cat([f_spatial, f_temporal], dim=1)
        fused = self.fusion(f_combined)
        logits = self.classifier(fused)
        return logits

# ------------------------------
# CSV Dataset
# ------------------------------
class CSVMentalHealthDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        features = torch.tensor(self.X[idx], dtype=torch.float32)
        seq = features.unsqueeze(0)  # 1D temporal input (1, seq_len)
        label = torch.tensor(self.y[idx], dtype=torch.long)
        return features, seq, label

# ------------------------------
# Training & evaluation
# ------------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for spatial_feat, seq, label in loader:
        spatial_feat, seq, label = spatial_feat.to(DEVICE), seq.to(DEVICE), label.to(DEVICE)
        optimizer.zero_grad()
        logits = model(spatial_feat, seq)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * spatial_feat.size(0)
        preds = logits.argmax(1)
        correct += (preds == label).sum().item()
        total += spatial_feat.size(0)

    return total_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for spatial_feat, seq, label in loader:
            spatial_feat, seq, label = spatial_feat.to(DEVICE), seq.to(DEVICE), label.to(DEVICE)
            logits = model(spatial_feat, seq)
            loss = criterion(logits, label)
            total_loss += loss.item() * spatial_feat.size(0)
            preds = logits.argmax(1)
            correct += (preds == label).sum().item()
            total += spatial_feat.size(0)
    return total_loss/total, correct/total

# ------------------------------
# Main
# ------------------------------
def main():
    # Load CSV
    csv_file = "/content/drive/MyDrive/Colab Notebooks/archive (22)/selected_features.csv"
    df = pd.read_csv(csv_file)
    feature_cols = df.columns[:-1]
    label_col = "Expert Diagnose"

    X = df[feature_cols].values.astype(np.float32)

    # Encode labels to 0-based integers
    le = LabelEncoder()
    y = le.fit_transform(df[label_col].values)

    NUM_CLASSES = len(np.unique(y))  # dynamically set

    # Train/Val split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
    train_ds = CSVMentalHealthDataset(X_train, y_train)
    val_ds = CSVMentalHealthDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

    # Model
    model = NeuroVisionNet(input_dim=X.shape[1], num_classes=NUM_CLASSES).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(1, EPOCHS+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch}: Train loss {tr_loss:.4f}, acc {tr_acc*100:.2f}% | Val loss {val_loss:.4f}, acc {val_acc*100:.2f}%")

    # Final validation accuracy
    val_loss, val_acc = evaluate(model, val_loader, criterion)


    # Test one batch for probabilities
    spatial_feat, seq, label = next(iter(val_loader))
    spatial_feat, seq = spatial_feat.to(DEVICE), seq.to(DEVICE)
    with torch.no_grad():
        logits = model(spatial_feat, seq)
        probs = F.softmax(logits, dim=1)

# Load the npy file
metrics = np.load("/content/drive/MyDrive/Colab Notebooks/archive (22)/behavioral_metrics.npy", allow_pickle=True).item()
if __name__ == "__main__":
    main()

# Print loaded metrics
for key, value in metrics.items():
    print(f"{key}: {value}")


Epoch 1: Train loss 1.3618, acc 32.29% | Val loss 1.3380, acc 54.17%
Epoch 2: Train loss 1.2969, acc 59.38% | Val loss 1.2695, acc 66.67%
Epoch 3: Train loss 1.2328, acc 65.62% | Val loss 1.2001, acc 66.67%
Epoch 4: Train loss 1.1295, acc 72.92% | Val loss 1.1211, acc 70.83%
Epoch 5: Train loss 1.0328, acc 70.83% | Val loss 1.0303, acc 70.83%
Epoch 6: Train loss 0.9576, acc 72.92% | Val loss 0.9458, acc 70.83%
Epoch 7: Train loss 0.8385, acc 76.04% | Val loss 0.8608, acc 70.83%
Epoch 8: Train loss 0.7381, acc 80.21% | Val loss 0.7887, acc 75.00%
Epoch 9: Train loss 0.6501, acc 84.38% | Val loss 0.7218, acc 75.00%
Epoch 10: Train loss 0.5950, acc 80.21% | Val loss 0.6693, acc 70.83%
Epoch 11: Train loss 0.5409, acc 78.12% | Val loss 0.6270, acc 70.83%
Epoch 12: Train loss 0.5277, acc 76.04% | Val loss 0.5993, acc 75.00%
Epoch 13: Train loss 0.5017, acc 78.12% | Val loss 0.6175, acc 70.83%
Epoch 14: Train loss 0.4957, acc 76.04% | Val loss 0.5762, acc 70.83%
Epoch 15: Train loss 0.4678, 

# **EEG Signals**

# **Preprocessing**
# **Text-to-Embeddings Conversion Using BERT**

In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
import nltk
import re
import string
from nltk.corpus import stopwords

# -----------------------------------
# 0. Download NLTK stopwords
# -----------------------------------
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# -----------------------------------
# 1. Preprocessing function
# -----------------------------------
def preprocess_text(text):
    """Clean and normalize text data"""
    if not isinstance(text, str):
        return ""
    text = text.lower()  # lowercase
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # remove unwanted chars
    tokens = [word for word in text.split() if word not in stop_words]  # remove stopwords
    return " ".join(tokens)

# -----------------------------------
# 2. Load BERT model + tokenizer
# -----------------------------------
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# -----------------------------------
# 3. Embedding function (Eqns 3–6)
# -----------------------------------
def get_bert_embeddings(text, pooling="cls"):
    """Convert text into embeddings using BERT"""
    clean_text = preprocess_text(text)
    inputs = tokenizer(clean_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = bert_model(**inputs)

    last_hidden_state = outputs.last_hidden_state.squeeze(0)  # (seq_len, hidden_dim)
    cls_embedding = outputs.pooler_output.squeeze(0)          # CLS token embedding

    if pooling == "cls":
        return cls_embedding.numpy()
    elif pooling == "mean":
        return last_hidden_state.mean(dim=0).numpy()
    else:
        raise ValueError("Pooling must be 'cls' or 'mean'")

# -----------------------------------
# 4. Fusion with EEG embeddings (Eqn 7)
# -----------------------------------
def fuse_embeddings(eeg_embedding, text_embedding, w1=0.5, w2=0.5, bias=0.1):
    eeg_part = w1 * np.array(eeg_embedding)
    text_part = w2 * np.array(text_embedding) + bias
    return np.concatenate([eeg_part, text_part])

# -----------------------------------
# 5. Process CSV input
# -----------------------------------
def process_csv(input_csv, text_column, output_csv, pooling="cls", use_fusion=False, keep_columns=None):
    """
    input_csv: path to input CSV file
    text_column: name of the column containing text
    output_csv: path to save embeddings + label column
    pooling: 'cls' or 'mean'
    use_fusion: whether to fuse with EEG embeddings
    keep_columns: list of columns from the input CSV to also include in the output
    """
    df = pd.read_csv(input_csv)

    embeddings = []
    for idx, text in enumerate(df[text_column]):
        emb = get_bert_embeddings(str(text), pooling=pooling)

        if use_fusion:
            eeg_emb = np.random.rand(len(emb))  # Simulated EEG embedding
            emb = fuse_embeddings(eeg_emb, emb)

        embeddings.append(emb)

    # Convert list of embeddings to DataFrame
    emb_df = pd.DataFrame(embeddings)

    # Add original column(s) back
    if keep_columns is None:
        emb_df[text_column] = df[text_column]
    else:
        for col in keep_columns:
            emb_df[col] = df[col]

    # Save to CSV
    emb_df.to_csv(output_csv, index=False)
    print(f"✅ Saved embeddings + labels to {output_csv}")

# -----------------------------------
# Example usage
# -----------------------------------
if __name__ == "__main__":
    input_csv = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/stress depression final.csv"
    output_csv = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/text_embeddings.csv"
    text_column = "Stress_level"   # column containing text

    # Keep additional output columns (optional)
    keep_cols = ["Stress_level"]   # or e.g. ["Expert Diagnose", "Patient ID", "Age"]

    # Process CSV → Save embeddings
    process_csv(input_csv, text_column, output_csv, pooling="cls", use_fusion=True, keep_columns=keep_cols)

    # Load and check
    df = pd.read_csv(output_csv)
    print(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Saved embeddings + labels to /content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/text_embeddings.csv
          0         1         2         3         4         5         6  \
0  0.146175  0.248382  0.094355  0.217579  0.209754  0.071075  0.117518   
1  0.466969  0.048016  0.051521  0.288556  0.276810  0.260594  0.138815   
2  0.315906  0.378557  0.227388  0.070926  0.034317  0.269473  0.499305   
3  0.002262  0.144946  0.123559  0.076075  0.350515  0.321943  0.252439   
4  0.288271  0.130950  0.373306  0.065946  0.229614  0.236475  0.155691   

          7         8         9  ...     1527      1528      1529      1530  \
0  0.013684  0.230360  0.487249  ...  0.14967  0.578413  0.572492 -0.260953   
1  0.200286  0.055650  0.235529  ...  0.14967  0.578413  0.572492 -0.260953   
2  0.111251  0.299194  0.119896  ...  0.14967  0.578413  0.572492 -0.260953   
3  0.373419  0.423112  0.400979  ...  0.14967  0.578413  0.572492 -0.260953   
4  0.389057  0.160364  0.008237  ...  0.14

# **Feature extraction**
# **BiLSTM-CNN Algorithm**

In [2]:
# -----------------------------
# Import Libraries
# -----------------------------
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# -----------------------------
# 1. Load CSV
# -----------------------------
csv_file = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/text_embeddings.csv"
df = pd.read_csv(csv_file)

# Separate features and labels
y = df['Stress_level'].values
X = df.drop('Stress_level', axis=1).values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -----------------------------
# 2. Infer number of channels and time steps
# -----------------------------
num_channels = 14  # adjust for your EEG
total_features = X.shape[1]

if total_features % num_channels != 0:
    time_steps = total_features // num_channels
    X = X[:, :num_channels*time_steps]
else:
    time_steps = total_features // num_channels

print(f"Num channels: {num_channels}, Time steps: {time_steps}, X shape: {X.shape}")

X = X.reshape(-1, num_channels, time_steps)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# -----------------------------
# 3. EEG Dataset Class
# -----------------------------
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = EEGDataset(X, y_encoded)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# -----------------------------
# 4. BiLSTM-CNN Model (Feature Extractor)
# -----------------------------
class BiLSTM_CNN_FeatureExtractor(nn.Module):
    def __init__(self, input_channels, time_steps, hidden_dim, cnn_out_channels):
        super(BiLSTM_CNN_FeatureExtractor, self).__init__()

        # BiLSTM
        self.bilstm = nn.LSTM(input_size=input_channels, hidden_size=hidden_dim,
                              num_layers=1, batch_first=True, bidirectional=True)

        # CNN
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)

    def forward(self, x):
        # BiLSTM features
        x_lstm = x.permute(0, 2, 1)  # (batch, time_steps, channels)
        lstm_out, _ = self.bilstm(x_lstm)
        lstm_feat = lstm_out[:, -1, :]  # last time step

        # CNN features
        cnn_out = self.conv1(x)
        cnn_out = self.relu(cnn_out)
        cnn_out = self.pool(cnn_out)
        cnn_feat = cnn_out.view(cnn_out.size(0), -1)

        # Concatenate features
        fused_feat = torch.cat((lstm_feat, cnn_feat), dim=1)
        return fused_feat

hidden_dim = 64
cnn_out_channels = 32
model = BiLSTM_CNN_FeatureExtractor(input_channels=num_channels, time_steps=time_steps,
                                    hidden_dim=hidden_dim, cnn_out_channels=cnn_out_channels)

model.eval()

# -----------------------------
# 5. Extract Features and Save to CSV
# -----------------------------
all_features = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in loader:
        feats = model(X_batch)
        all_features.append(feats.numpy())
        all_labels.append(y_batch.numpy())

all_features = np.vstack(all_features)
all_labels = np.concatenate(all_labels)

# Combine features and labels
df_features = pd.DataFrame(all_features)
df_features['Stress_level'] = all_labels

# Save to CSV
output_csv = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/extraction_features.csv"
df_features.to_csv(output_csv, index=False)
print(f"Feature extraction completed. Saved to {output_csv}")

df_features

Num channels: 14, Time steps: 109, X shape: (100, 1526)
Feature extraction completed. Saved to /content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/extraction_features.csv


,0,1,2,3,4,5,6,7,8,9,...,1847,1848,1849,1850,1851,1852,1853,1854,1855,Stress_level
0,0.091677,0.058372,0.003939,0.039251,0.064874,0.029254,0.022749,0.108396,0.172585,0.025188,...,0.078061,0.453214,0.030447,0.173015,0.334764,0.000000,0.064304,0.937713,0.772895,0
1,0.152873,-0.028041,-0.099483,-0.078628,0.067403,0.142819,-0.080713,-0.106415,0.136727,0.201685,...,0.258374,1.048739,0.266284,0.228701,0.441502,0.774047,0.234257,0.665802,0.543874,0
2,0.016224,0.074718,0.022343,0.043940,0.173617,-0.007722,0.047211,0.085951,0.075537,0.065307,...,0.634703,0.276349,0.000000,0.000000,0.380868,0.106865,0.715298,0.088844,0.280119,0
3,0.052530,0.036977,-0.013193,0.008240,0.129612,0.042955,-0.038848,-0.073606,0.062133,0.157726,...,0.588710,0.477186,0.315150,0.305230,0.553598,0.678465,0.271253,0.091649,0.105751,0
4,0.184467,0.060306,-0.060478,-0.100389,0.109222,0.074871,0.027741,0.007308,0.083230,0.208729,...,0.049420,0.481573,0.331997,0.405178,0.874837,0.985192,0.000000,0.000000,0.030483,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.030189,0.013424,-0.040953,0.060748,0.136641,0.092085,-0.104140,-0.007985,0.042168,0.006862,...,0.497736,0.696791,0.802114,0.177031,0.000000,0.561132,0.051316,0.000000,0.203519,0
96,0.010855,0.105179,0.077318,0.033615,0.149694,-0.033840,-0.000892,0.004237,0.071040,0.099167,...,0.131602,0.097526,0.087990,0.602738,1.222575,0.677637,0.707181,0.523318,0.161043,0
97,-0.030023,0.032296,0.054605,0.009450,0.091993,-0.046941,0.042133,0.031248,-0.008644,0.167853,...,0.549755,0.294287,0.110707,0.846226,0.677149,0.401797,0.674281,0.582182,0.352144,0
98,0.154424,0.036427,-0.026841,-0.095188,0.098195,0.076485,0.015579,0.043387,0.182811,0.238035,...,0.000000,0.234500,0.000000,0.138032,0.554019,0.467414,0.308466,0.459221,1.147918,0


# **Feature selection via HAGWO**

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# -----------------------------
# 1. Load CSV
# -----------------------------
csv_file = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/extraction_features.csv"
df = pd.read_csv(csv_file)

# Separate features and label
X = df.drop('Stress_level', axis=1).values
y = df['Stress_level'].values

# Optional: normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -----------------------------
# 2. HAGWO Class
# -----------------------------
class HAGWO:
    def __init__(self, X, y, num_ants=20, num_iterations=10, min_features=5):
        self.X = X
        self.y = y
        self.M = num_ants
        self.T = num_iterations
        self.N = X.shape[1]
        self.tau = np.ones(self.N)
        self.eta = np.random.rand(self.N)
        self.rho = 0.1
        self.q = 1.0
        self.min_features = min_features

    def fitness(self, subset):
        # Replace with real classifier if available
        if subset.sum() == 0:
            return 0
        return subset.sum() / len(subset)

    def aco_exploration(self):
        ant_subsets = []
        for _ in range(self.M):
            probs = (self.tau ** 1) * (self.eta ** 2)
            probs = probs / probs.sum()
            subset = np.random.rand(self.N) < probs
            # Ensure minimum features
            if subset.sum() < self.min_features:
                indices = np.random.choice(self.N, self.min_features, replace=False)
                subset[indices] = 1
            ant_subsets.append(subset.astype(int))
        return np.array(ant_subsets)

    def update_pheromones(self, ant_subsets):
        delta_tau = np.zeros(self.N)
        for subset in ant_subsets:
            fit = self.fitness(subset)
            delta_tau += self.q * subset * fit
        self.tau = (1 - self.rho) * self.tau + delta_tau

    def mgwo_exploitation(self, top_subsets):
        alpha, beta, delta = top_subsets[:3]
        new_subsets = []
        a = 2.0
        for subset in top_subsets:
            r1, r2, r3 = np.random.rand(3, self.N)
            D_alpha = np.abs(r1 * alpha - subset)
            D_beta = np.abs(r2 * beta - subset)
            D_delta = np.abs(r3 * delta - subset)
            x1 = alpha - a * D_alpha
            x2 = beta - a * D_beta
            x3 = delta - a * D_delta
            new_subset = (x1 + x2 + x3) / 3
            new_subset = (new_subset > 0.5).astype(int)
            # Ensure minimum features
            if new_subset.sum() < self.min_features:
                indices = np.random.choice(self.N, self.min_features, replace=False)
                new_subset[indices] = 1
            new_subsets.append(new_subset)
        return np.array(new_subsets)

    def select_features(self):
        best_subset = np.zeros(self.N)
        best_score = -np.inf

        for t in range(self.T):
            ant_subsets = self.aco_exploration()
            self.update_pheromones(ant_subsets)

            fitnesses = np.array([self.fitness(s) for s in ant_subsets])
            top_idx = fitnesses.argsort()[-max(1, self.M//10):][::-1]
            top_subsets = ant_subsets[top_idx]

            new_subsets = self.mgwo_exploitation(top_subsets)

            for subset in new_subsets:
                score = self.fitness(subset)
                if score > best_score:
                    best_score = score
                    best_subset = subset

        # Final check: ensure minimum features
        if best_subset.sum() < self.min_features:
            indices = np.random.choice(self.N, self.min_features, replace=False)
            best_subset[indices] = 1

        return best_subset

# -----------------------------
# 3. Run HAGWO
# -----------------------------
hagwo = HAGWO(X, y, num_ants=30, num_iterations=20, min_features=5)
selected_mask = hagwo.select_features()

# -----------------------------
# 4. Save reduced dataset with original column names
# -----------------------------
X_selected = X[:, selected_mask==1]
feature_columns = df.drop('Stress_level', axis=1).columns
selected_columns = feature_columns[selected_mask==1]

df_selected = pd.DataFrame(X_selected, columns=selected_columns)
df_selected['Stress_level'] = y  # Add label column

output_csv = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/selected_features.csv"
df_selected.to_csv(output_csv, index=False)

print(f"Selected features saved to {output_csv}")
print("Number of selected features:", X_selected.shape[1])
print("Selected feature columns:", list(selected_columns))
df_selected


Selected features saved to /content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/selected_features.csv
Number of selected features: 5
Selected feature columns: ['580', '1006', '1386', '1440', '1736']


,580,1006,1386,1440,1736,Stress_level
0,0.391204,-0.403002,0.522287,-0.463980,-0.353716,0
1,-0.468476,-1.205977,3.791587,-0.515631,-0.579872,0
2,0.348986,0.386246,-1.202513,3.218590,-0.527044,0
3,-1.035805,-1.205977,-0.224574,-0.793694,-0.506195,0
4,0.919286,-1.205977,-0.712825,-0.793694,-0.341370,0
...,...,...,...,...,...,...
95,3.586889,1.080295,0.687161,0.081121,-0.877114,0
96,1.073958,0.388732,-1.037543,0.056370,1.411359,0
97,1.531180,0.141969,-1.202513,0.420431,0.236064,0
98,0.023534,1.020462,-0.170347,-0.793694,-1.176016,0


# **classification**

In [8]:
"""
NeuroVisionNet: Multimodal mental health diagnosis model
- CSV numerical features as spatial + temporal inputs
- Label column: "Expert Diagnose"
- Prints training & validation accuracy
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ------------------------------
# Config
# ------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SPATIAL_FEATURE_DIM = 512
TEMPORAL_FEATURE_DIM = 256
FUSION_HIDDEN = 256
DROPOUT = 0.3
LR = 1e-4
BATCH_SIZE = 8
EPOCHS = 100

# ------------------------------
# Temporal branch: T-CNN (1D Conv)
# ------------------------------
class TemporalCNN(nn.Module):
    def __init__(self, in_channels=1, out_dim=TEMPORAL_FEATURE_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, 32, 7, padding=3), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 5, padding=2), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(128, out_dim)

    def forward(self, x):
        y = self.net(x)
        y = y.view(y.size(0), -1)
        return self.fc(y)

# ------------------------------
# NeuroVisionNet
# ------------------------------
class NeuroVisionNet(nn.Module):
    def __init__(self, input_dim, num_classes, dropout=DROPOUT):
        super().__init__()
        self.spatial = nn.Linear(input_dim, SPATIAL_FEATURE_DIM)
        self.temporal = TemporalCNN(out_dim=TEMPORAL_FEATURE_DIM)
        self.fusion = nn.Sequential(
            nn.Linear(SPATIAL_FEATURE_DIM + TEMPORAL_FEATURE_DIM, FUSION_HIDDEN),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(FUSION_HIDDEN, FUSION_HIDDEN//2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.classifier = nn.Linear(FUSION_HIDDEN//2, num_classes)

    def forward(self, spatial_feat, seq):
        f_spatial = self.spatial(spatial_feat)
        f_temporal = self.temporal(seq)
        f_combined = torch.cat([f_spatial, f_temporal], dim=1)
        fused = self.fusion(f_combined)
        logits = self.classifier(fused)
        return logits

# ------------------------------
# CSV Dataset
# ------------------------------
class CSVMentalHealthDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        features = torch.tensor(self.X[idx], dtype=torch.float32)
        seq = features.unsqueeze(0)  # 1D temporal input (1, seq_len)
        label = torch.tensor(self.y[idx], dtype=torch.long)
        return features, seq, label

# ------------------------------
# Training & evaluation
# ------------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for spatial_feat, seq, label in loader:
        spatial_feat, seq, label = spatial_feat.to(DEVICE), seq.to(DEVICE), label.to(DEVICE)
        optimizer.zero_grad()
        logits = model(spatial_feat, seq)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * spatial_feat.size(0)
        preds = logits.argmax(1)
        correct += (preds == label).sum().item()
        total += spatial_feat.size(0)

    return total_loss/total, correct/total

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for spatial_feat, seq, label in loader:
            spatial_feat, seq, label = spatial_feat.to(DEVICE), seq.to(DEVICE), label.to(DEVICE)
            logits = model(spatial_feat, seq)
            loss = criterion(logits, label)
            total_loss += loss.item() * spatial_feat.size(0)
            preds = logits.argmax(1)
            correct += (preds == label).sum().item()
            total += spatial_feat.size(0)
    return total_loss/total, correct/total

# ------------------------------
# Main
# ------------------------------
def main():
    # Load CSV
    csv_file = "/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/selected_features.csv"
    df = pd.read_csv(csv_file)
    feature_cols = df.columns[:-1]
    label_col = "Stress_level"

    X = df[feature_cols].values.astype(np.float32)

    # Encode labels to 0-based integers
    le = LabelEncoder()
    y = le.fit_transform(df[label_col].values)

    NUM_CLASSES = len(np.unique(y))  # dynamically set

    # Train/Val split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
    train_ds = CSVMentalHealthDataset(X_train, y_train)
    val_ds = CSVMentalHealthDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

    # Model
    model = NeuroVisionNet(input_dim=X.shape[1], num_classes=NUM_CLASSES).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(1, EPOCHS+1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch}: Train loss {tr_loss:.4f}, acc {tr_acc*100:.2f}% | Val loss {val_loss:.4f}, acc {val_acc*100:.2f}%")

    # Final validation accuracy
    val_loss, val_acc = evaluate(model, val_loader, criterion)


    # Test one batch for probabilities
    spatial_feat, seq, label = next(iter(val_loader))
    spatial_feat, seq = spatial_feat.to(DEVICE), seq.to(DEVICE)
    with torch.no_grad():
        logits = model(spatial_feat, seq)
        probs = F.softmax(logits, dim=1)

# Load the npy file
metrics = np.load("/content/drive/MyDrive/Colab Notebooks/F_Relax_A_feature.csv/eeg_metrics.npy", allow_pickle=True).item()
if __name__ == "__main__":
    main()

# Print loaded metrics
for key, value in metrics.items():
    print(f"{key}: {value}")


Epoch 1: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 2: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 3: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 4: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 5: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 6: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 7: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 8: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 9: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 10: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 11: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 12: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 13: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
Epoch 14: Train loss 0.0000, acc 100.00% | Val loss 0.0000, acc 100.00%
E